In [ ]:
from dataparser import Data_parser
from label_encoder_np import Label_encoder_v2,get_feat_shape,get_bbox_size,generate_anchors
from ggmodel import GG_SSD
from ggloss import SSD_Loss_v2
from box_decoder import Box_decoder,boxVisualization
import tensorflow as tf
import cv2


In [ ]:
#設置超參數
image_shape = (512,512)
classes = ['Cyan-Blue','Blue','Green','Red']
num_pred_layer = 5
Smin = 0.05
Smax = 0.8
steps = ([8,16,32,64,128])
#--------------------------------------------
sizes = get_bbox_size(image_shape,num_pred_layer,Smin,Smax)# 預設box大小
ratios =([2,0.5],
        [2,0.5],
        [2,0.5],
        [2,0.5],
        [2,0.5])   # 預設box長寬比
feat_shape = get_feat_shape(image_shape,steps)
SSD_decoder = Box_decoder(image_shape, feat_shape, sizes, ratios, steps)

In [ ]:
#讀取並解析XML
XML = Data_parser(r'C:\Users\User\GGjupyter\DeepLearning-Data\Butterfly\train',Classes = classes)
decoded_labels = XML.xml_parser()

In [ ]:
Anchors = generate_anchors(image_shape, 
                  feat_shape, 
                  sizes, 
                  ratios, 
                  steps)

In [ ]:
#encode bbox
y_train = Label_encoder_v2(decoded_labels,
                  image_shape, 
                  feat_shape, 
                  sizes, 
                  ratios,
                  steps,
                  Anchors)

print(f'shape of y_train = {y_train.shape}')

In [ ]:
#讀取影像
from skimage import io
import numpy as np


train = []
for File in decoded_labels:
    img = io.imread(r'C:\Users\User\GGjupyter\DeepLearning-Data\Butterfly\train'+'/'+File[0])[:,:,0:3]/255
    img = img.astype('float32')
    train.append(img)
x_train = np.asarray(train)

In [ ]:
for i in range(134):
    label_reshape = y_train[i].numpy()
    label_reshape = label_reshape.reshape([1] + y_train.shape[1:])
    label_reshape = tf.convert_to_tensor(label_reshape)
    print(i+1)
    img = x_train[i].copy()
    Boxes = SSD_decoder.label_box(label_reshape)
    print(Boxes)

    image = boxVisualization(Boxes,img)
    io.imshow(image)
    io.show()

In [ ]:
#建立Model及Loss
model = GG_SSD(len(classes),sizes,ratios)
loss = SSD_Loss_v2(len(classes))

In [ ]:
#compile
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.0012, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0005)
model.compile(optimizer=adam, loss=loss.compute_loss)

In [ ]:
def dataGenerator(x_train,y_train,batch_size):
    if len(x_train) != len(y_train):
        raise ValueError(f"length of x_train is {len(x_train)} but get {len(y_train)} as length of y_train")
    index = 0
    while index < len(x_train):
        if index + batch_size > len(x_train):
            index = 0
            shuffle = np.arange(len(x_train))
            np.random.shuffle(shuffle)
            x_train,y_train = x_train[shuffle],y_train[shuffle]
        yield x_train[index:index+batch_size],y_train[index:index+batch_size]
        index += batch_size

GGenerator = dataGenerator(x_train,y_train,4)

In [ ]:
#開始訓練
model.fit(GGenerator,epochs=100,steps_per_epoch = 100)

In [ ]:
for i in range(134):
    
    img = x_train[i].copy()
    img_reshape = img.reshape(1,512,512,3)
    result = model(img_reshape)

    Boxes = SSD_decoder(result)
    for box in Boxes:
        print(box[4])
    image = boxVisualization(Boxes,img)
    io.imshow(image)
    io.show()

In [ ]:
Boxes = SSD_decoder(result)
for box in Boxes:
    print(box[4])
image = boxVisualization(Boxes,img)
io.imshow(image)

In [ ]:
# def boxVisualization(Boxes,image,image_size = image_shape):
#     img = image.copy()
#     for box in Boxes:
#         ymin = box[0]*image_size[0]
#         xmin = box[1]*image_size[1]
#         ymax = box[2]*image_size[0]
#         xmax = box[3]*image_size[1]
#         img = cv2.rectangle(img,(int(xmin),int(ymin)),(int(xmax),int(ymax)),(15,235,80),2)
#     return img

In [ ]:
# DBOX = SSD_decoder.anchor_pred

In [ ]:
# img = np.zeros(shape = (512,512,3))
# for i,box in enumerate(DBOX):
#     if i%3 == 0 and i%7 == 0 and i%5 == 0 and i%13 == 0: 
#         ymin = box[0]*512
#         xmin = box[1]*512
#         ymax = box[2]*512
#         xmax = box[3]*512
#         img = cv2.rectangle(img,(int(xmin),int(ymin)),(int(xmax),int(ymax)),(15,235,80),1)

In [ ]:
# io.imshow(img)
# io.imsave('test.png',img)